In [1]:
import pandas as pd
import numpy as np
from numpy import array
import matplotlib.pyplot as plt
import math
import openpyxl
from statistics import * 

In [4]:
#This code will parse through the Peak_ID worksheet and generate another worksheet in the same file called "External_STD"
#with concentration and peak area data for C7, C8, ..., C17 arranged like so:
#
# Chain Length | Conc (mg/L) | Peak Area | Conc (mg/L) | Peak Area | Conc (mg/L) | Peak Area | Conc (mg/L) | Peak Area
# _____________________________________________________________________________________________________________________
#      C7      |    1000     |     #     |     500     |     #     |     100     |     #     |     25      |     #
#      C8      |    1000     |     #     |     500     |     #     |     100     |     #     |     25      |     #
#      ...     |    1000     |     #     |     500     |     #     |     100     |     #     |     25      |     #
#      C17     |    1000     |     #     |     500     |     #     |     100     |     #     |     25      |     #
#

#filename with GC data
file = 'GCData_for_Ryan_w_FaOH_STD_Data.xlsx'
#sheet name with GC data
sheetname = 'Peak_ID (3)'
#column number containing peak area data
area_col = 3
#column number containgin chain length ID
chain_col = 5
#type of external standard (FAOH or FAME) - assumes the following layout: "<type>-<concentration>" i.e. FAOH-100
alc_acid_ID = 'FAOH'
df = getExternalStdData(file, sheetname, area_col, chain_col, alc_acid_ID)
df

,Chain Length,Conc (mg/L),Peak Area,Conc (mg/L),Peak Area,Conc (mg/L),Peak Area,Conc (mg/L),Peak Area,Conc (mg/L),Peak Area
1,C7,10,19664,50,102211,100,1828,500,1177901,2000,2522844
2,C8,10,12086,50,96901,100,-,500,1232414,2000,2589475
3,C9,10,12220,50,109416,100,2363,500,1323017,2000,2747118
4,C10,10,11364,50,112460,100,2707,500,1295064,2000,2679104
5,C11,10,11354,50,121218,100,4028,500,1342950,2000,2783594
6,C12,10,11671,50,131328,100,6151,500,1417179,2000,2929293
7,C13,10,11474,50,140772,100,8154,500,1494950,2000,3076296
8,C14,10,15509,50,143713,100,14350,500,1547138,2000,3113534
9,C15,10,47410,50,158776,100,37876,500,1660713,2000,3210020
10,C16,10,136275,50,211310,100,139815,500,1526528,2000,3075718


In [5]:
def getExternalStdData(file, sheetname, area_col, chain_col, alc_acid_ID):
    col = [area_col, chain_col]
    wb = openpyxl.load_workbook(file)
    sheet = wb[sheetname]
    #create dataframe from relevant columns only
    df = pd.read_excel(file, sheetname=sheetname, usecols = col, header = None)
    #convert dataframe to array to allow array traversals
    dfList = df.values.tolist()
    #Master_dict will hold area and concentration data 
    Master_dict = {'C7':[], 'C8':[], 'C9':[], 'C10':[], 'C11':[], 'C12':[],
                 'C13':[], 'C14':[], 'C15':[], 'C16':[], 'C17':[]}

    # creates list of external standard title indicies (location of titles)
    alc_acid_indicies = []
    for i in range(len(dfList)):
        name = dfList[i][1]
        if name[0:name.find('-')] == alc_acid_ID:
            alc_acid_indicies.append(i)
    
    conc_array = []
    # start adding to master matrix here using indicies of titles
    for i in alc_acid_indicies:
        #find the position right before the start of a new set of data as indicated by a string
        endindex = i
        for k in range(len(dfList[i:len(dfList)])):
            endindex+=1
            if(isinstance(dfList[endindex][0],str)):
                break
        #parse title to find concentration (gets the number after '-' in the title) and add to conc_array and sort
        conc = dfList[i][1][dfList[i][1].find('-')+1:len(dfList[i][1])]
        conc_array.append(int(conc))
        conc_array.sort()
        #for each chain, add the concentration and area data to the Master dictionary
        for j in range(endindex-i):
            if(isinstance(dfList[i+j][0], int)):
                conc_and_area = [conc, dfList[i+j][0]]
                Master_dict[dfList[i+j][1]].append(conc_and_area)
    
    #create master matrix holding arrays of area values
    num_conc, num_chain = len(conc_array), len(Master_dict);
    Master = [[0 for x in range(num_conc*2+1)] for y in range(num_chain+1)]
    
    #populate column titles
    Master[0][0] = 'Chain Length'
    for i in range(1, num_conc*2+1):
        if i%2==0:
            Master[0][i] = 'Peak Area'
        else:
            Master[0][i] = 'Conc (mg/L)'
    
    #create matrix with desired layout as seen in Cell 2
    chain_iterator = 1
    for chain in Master_dict:
        conc_iterator = 1
        #label first column with chain
        Master[chain_iterator][0] = chain
        for conc in conc_array:
            area = []
            #search for correct area data. If found, add to area array, even if there is multiple data
            for pair in Master_dict[chain]:
                if int(pair[0]) == conc:
                    area.append(pair[1])
            #this accounts for double-counting areas (only picks the largest one)
            area.sort(reverse = True)
            #this accounts for no area data
            if len(area) == 0:
                area = ['-']
            #add to the master matrix
            Master[chain_iterator][conc_iterator] = conc
            conc_iterator+=1
            Master[chain_iterator][conc_iterator] = area[0]
            #move on to the next concentration
            conc_iterator+=1
        #move on to the next chain
        chain_iterator+=1
    #create new dataframe from Master array
    excelData = pd.DataFrame(data=Master, index=None)
    #Replace header w/ top row for aesthetic
    new_header = excelData.iloc[0]
    excelData = excelData[1:]
    excelData.columns = new_header
    #write to the excel file
    writer = pd.ExcelWriter(file, engine = 'openpyxl')
    writer.book = wb
    excelData.to_excel(writer, sheet_name = "External_STD", index = False)
    wb.save(filename = file)
    return excelData